In [21]:
import os, json, pandas, re
from xml.etree import ElementTree as et

In [22]:
class document:

    def __init__(self, folder):
        
        self.folder = folder
        return
    
    def read(self):

        group = {
            'path' : [], 
            'original content': [],
            'character size':[],
            'word size':[],
            'sentence size':[],
        }
        pass

        cache = {}
        for index, name in enumerate(os.listdir(self.folder), 1):

            path = os.path.join(self.folder, name)
            group['path'] += [path]
            if('xml' in name):

                data = et.parse(path)
                root = data.getroot()

                ##  擷取所有的字串。
                content = "".join([e.text for e in root.iter()])
                group['original content'] = [content]

                text = ""
                for article in root.findall("PubmedArticle"):

                    abstract = article.find("MedlineCitation").find("Article").find("Abstract")
                    text += "".join([e.text for e in abstract.findall("AbstractText")])
                    pass

                character = re.sub(r'[^{}{}{}]+'.format(string.punctuation, string.digits, string.ascii_letters),'', text)
                sentence  = text.split(".")[:-1]
                word = re.sub("\s+", " ", re.sub(r'[{}]+'.format(string.punctuation), " ", text)).splot(" ")
                group['character size'] += [len(character)]
                group['sentence size'] += [len(sentence)]
                group['word size'] += [len(word)]
                pass

            if('json' in name):

                with open(path, 'r') as paper:

                    data = json.load(paper)
                    pass

                content = []
                text = []
                for i in data:

                    for k, v in i.items():

                        if(k=='tweet_text'):
                            
                            text += [v]
                            pass

                        content += [v]        
                        pass

                    pass
                
                content = "".join(content)
                text = ''.join(text)
                character = re.sub(r'[^{}{}{}]+'.format(string.punctuation, string.digits, string.ascii_letters),'', text)
                word = re.sub("\s+", " ", re.sub(r'[{}]+'.format(string.punctuation), " ", text)).splot(" ")
                sentence  = text.split(".")[:-1]
                group['original content'] += [content]
                group['character size'] += [len(character)]
                group['sentence size'] += [len(sentence)]
                group['word size'] += [len(word)]
                pass
            
            number = index
            pass
        
        self.data = pandas.DataFrame(group)
        print("load {} files...".format(number))
        return
    
    def search(self, keyword):

        output = {
            "file":[],
            "exist":[],
            "number":[]
        }
        pass
    
        for _, item in self.data.iterrows():

            output['file'] += [item['file']]
            output['exist'] += [bool(re.search(keyword, item['content']))]
            output['number'] += [len(re.findall(keyword, item['content']))]
            pass
        
        output = pandas.DataFrame(output)
        return(output)

    pass

In [23]:
doc = document(folder = 'demo_data')
doc.read()

load 6 files...


In [24]:
doc.data

,file,content,character,word,word stem (unique),sentence,paragraph
0,test6.json,SWHealth NSW Health https://twitter.com/NSWHea...,456,63,50,7,1
1,test5.json,regAbbott_TX Greg Abbott https://twitter.com/G...,449,55,47,11,1
2,test4.json,andPaul Senator Rand Paul https://twitter.com/...,360,47,42,4,1
3,test1.xml,33053381 2020 12 21 2021 01 10 1879-0712 889 2...,3600,473,267,100,1
4,test2.xml,33342635 2021 01 27 2021 05 21 1873-2518 39 4 ...,8794,1176,560,278,1
5,test3.xml,32512530 2020 12 10 2021 01 10 1876-2026 53 20...,4779,618,337,155,1


In [11]:
doc.search(keyword='Wang')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,False,0
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0
